In [28]:
import pandas as pd
import json 
import os

## Phases of play
### How to obtain these 4 phases ?

![](../assets/images/Phases_of_play.png)

Pistes : 
- https://www.statsperform.com/resource/phases-of-play-an-introduction/ 


In [29]:
data_directory = '../data/StatsBomb'
df_events = list()

## Concaténation de tous les matchs 
for file in os.listdir(data_directory) :
    # Sélection des données d'évènements
    if 'events.json' in file : 
        match_events_path = os.path.join(data_directory, file)
    else :
        continue
    # Lecture du json
    with open(match_events_path) as json_file:
        match_events = json.load(json_file)
    # Ajout du dataframe obtenu
    df_events.append(pd.read_json(match_events_path))
    break
df_events = pd.concat(df_events)

In [30]:
df_events.columns

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'type',
       'possession', 'possession_team', 'play_pattern', 'obv_for_after',
       'obv_for_before', 'obv_for_net', 'obv_against_after',
       'obv_against_before', 'obv_against_net', 'obv_total_net', 'team',
       'duration', 'tactics', 'related_events', 'player', 'position',
       'location', 'pass', 'carry', 'ball_receipt', 'shot', 'goalkeeper',
       'under_pressure', 'counterpress', 'ball_recovery', 'dribble', 'duel',
       'out', 'clearance', 'off_camera', 'interception', 'foul_won',
       'foul_committed', 'block', 'substitution', 'miscontrol',
       'injury_stoppage'],
      dtype='object')

In [31]:
df_events.loc[:, 'play_pattern_id'] = df_events.play_pattern.apply(lambda x: x['id'])
df_events.loc[:, 'possession_team_id'] = df_events.possession_team.apply(lambda x: x['id'])
df_events.loc[:, 'team_id'] = df_events.team.apply(lambda x: x['id'])
df_events.loc[:, 'type_id'] = df_events.type.apply(lambda x: x['id'])

df_events.loc[~df_events.player.isna(), 'player_id'] = df_events.loc[~df_events.player.isna(), 'player'].apply(lambda x: x['id'])

df_events.loc[~df_events.location.isna(), 'x'] = df_events.loc[~df_events.location.isna(), 'location'].apply(lambda x : x[0])
df_events.loc[~df_events.location.isna(), 'y'] = df_events.loc[~df_events.location.isna(), 'location'].apply(lambda x : x[1])

In [32]:
min_possession_length = 3

# On ne considère uniquement les séquences de longueur supérieure à 3 ? 
# Longueur d'une possession
df_events.loc[:, 'possession_length'] = df_events.groupby('possession').id.transform('count')
df_events = df_events[df_events.possession_length >= min_possession_length]

# Suppression des coups de pieds arrétés et corner
df_events = df_events[~df_events.play_pattern_id.isin((2, 3))]

# On ne s'intéresse qu'à ManCity ? 
df_events = df_events[df_events.possession_team_id == 746]

In [33]:
# Un besoin de normaliser le sens du terrain avec le fichier FAWSL_22_23

In [66]:
play_phase = {
    1 : 'Build-up play',
    2 : 'Counter-attacking',
    3 : 'Final third penetration and finishing'
}

# Build-up play
# Toutes les actions qui débutent dans la moitié de terrain de l'équipe
df_events.loc[df_events.x < 70, 'play_phase_id'] = 1

# Final third penetration and finishing
# Action dans le dernier tiers 
df_events.loc[df_events.x > 80, 'play_phase_id'] = 3

# Counter-attacking
# L'id 6 de play_pattern tague directement les contres-attaques
df_events.loc[df_events.play_pattern_id == 6, 'play_phase_id'] = 2

## Viz

In [35]:
import plotly.graph_objects as go
import sys

sys.path.insert(0, '../')
from scripts.viz import create_field

In [67]:
df_events_finishing = df_events[df_events.play_phase_id == 1]

# Actions à au moins deux passes
df_events_finishing.possesion_length = df_events_finishing.groupby('possession').id.transform('count')
df_events_finishing = df_events_finishing[df_events_finishing.possesion_length >= 2]

# Actions uniquement de Man City
df_events_finishing = df_events_finishing[df_events_finishing.team_id == 746]

# Position moyenne des joueurs
player_position = df_events_finishing.groupby(['player_id'])[['x', 'y']].mean().reset_index()

# Passes et valeurs
pass_events = df_events_finishing[df_events_finishing.type_id == 30] 
pass_events.loc[:, 'recept_player_id'] = pass_events['pass'].apply(lambda x : x['recipient']['id'] if 'recipient' in x else None)

pass_events = pass_events.merge(player_position, right_on = 'player_id', left_on = 'player_id')
pass_events = pass_events.merge(player_position, right_on = 'player_id', left_on = 'recept_player_id')

/var/folders/pd/5qcm5zrx0fn0j7pq93xczxwhsmpxkh/T/ipykernel_49229/2785464161.py:4: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

/var/folders/pd/5qcm5zrx0fn0j7pq93xczxwhsmpxkh/T/ipykernel_49229/2785464161.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [68]:
fig = create_field()


# Define the nodes and edges of the graph
nodes = [
    go.Scatter(x=player_position.x, y = player_position.y, mode="markers", marker=dict(size=15, color="blue")),
]

edges = [
    go.Scatter(x=[x1, x2], y=[y1, y2], mode="lines", line=dict(width=2, color="red"))
    
    for x1, y1, x2, y2 in zip(pass_events.x_y, pass_events.y_y, pass_events.x, pass_events.y)
]

fig.add_traces(nodes + edges)
fig.update(layout_showlegend=False)
fig.show()